This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/robot.html).  I recommend having both windows open, side-by-side!

In [ ]:
import os
import urllib
from pathlib import Path
from zipfile import ZipFile

from pydrake.all import ModelVisualizer, PackageMap, Simulator, StartMeshcat
from tqdm.notebook import tqdm

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

In [ ]:
def AddRobocasaRemote(package_map):
    package_map.AddRemote(
        package_name="robocasa",
        params=PackageMap.RemoteParams(
            urls=[
                f"https://github.com/robocasa/robocasa/archive/1370b9e0f747d84fb21ed29bacefb1654865301b.zip"
            ],
            sha256=("a7218ed369936f96b19467eee5038870b14c2b7f91a9d6108591394ed074b337"),
            strip_prefix="robocasa-1370b9e0f747d84fb21ed29bacefb1654865301b/robocasa/",
        ),
    )


def DownloadRobocasaKitchenAssets():
    package_map = PackageMap()
    AddRobocasaRemote(package_map)
    # This will force the download if it hasn't been done before.
    robocasa_path = package_map.GetPath("robocasa")

    # The asset registry dictionary is copied from
    # https://github.com/robocasa/robocasa/blob/main/robocasa/scripts/download_kitchen_assets.py
    # with robocasa_path updated.
    DOWNLOAD_ASSET_REGISTRY = dict(
        fixtures=dict(
            message="Downloading fixtures",
            url="https://utexas.box.com/shared/static/956d0w2ucqs7d3eors1idsohgum57nli.zip",
            folder=os.path.join(robocasa_path, "models/assets/fixtures"),
            check_folder_exists=False,
        ),
    )

    def show_progress(block_num, block_size, total_size):
        if pbar.total is None:
            pbar.total = total_size
        pbar.update(block_size)

    for name, info in DOWNLOAD_ASSET_REGISTRY.items():
        with tqdm(unit="B", unit_scale=True, miniters=1, desc=info["message"]) as pbar:
            filename, headers = urllib.request.urlretrieve(
                info["url"], reporthook=show_progress
            )
        folder = Path(info["folder"]).parent
        with ZipFile(filename, "r") as zip_ref:
            zip_ref.extractall(path=folder)
        os.remove(filename)


# You'll only want to run this once.
# TODO(russt): Update this to MaybeDownloadRobocasaKitchenAssets.
# DownloadRobocasaKitchenAssets()

# Browse the assets

First we'll use the ModelVisualizer to inspect the models that are available [here](https://github.com/robocasa/robocasa/tree/1370b9e0f747d84fb21ed29bacefb1654865301b/robocasa).

In [ ]:
visualizer = ModelVisualizer(meshcat=meshcat)
AddRobocasaRemote(visualizer.parser().package_map())
visualizer.AddModels(
    url="package://robocasa/models/assets/fixtures/accessories/knife_blocks/dark_wood/model.xml"
)
visualizer.Run(loop_once=True)
meshcat.DeleteAddedControls()